In [27]:
import numpy as np
from math import fabs, tan, sin, cos

In [28]:
f = lambda x: tan(3*x) + 0.4 - x*x
f_1 = lambda x: 3/(1+x*x) - 2*x
Sistema = lambda x,y: [sin(y+2) - x - 1.5, y + cos(x-2) - 0.5]
Sistema1 = lambda a,x,y: [a*sin(y+2) - x - 1.5, y + a*cos(x-2) - 0.5]

In [29]:
class Matrix:
    
    A = [[]]
    def __init__(self, dim):
        self.dim1 = dim[0]
        self.dim2 = dim[1]
        self.A = [[0]*self.dim2 for i in range(self.dim1)]
        
    def __getitem__(self, i):
        return self.A[i]
    
    def __eq__(self, B):
        assert(self.dim1 == B.dim1 and self.dim2 == B.dim2)
        for i in range(self.dim1):
            for j in range(self.dim2):
                if fabs(self.A[i][j] - B[i][j]) > 1e-9:
                    return False
        return True
    
    def dim(self):
        return (self.dim1,self.dim2)
    
    def __add__(self, B):
        assert((self.dim1,self.dim2) == B.dim())
        result = Matrix((self.dim1,self.dim2))
        for i in range(self.dim1):
            for j in range(self.dim2):
                result[i][j] = self.A[i][j] + B[i][j]
        return result
    
    def __sub__(self, B):
        assert((self.dim1,self.dim2) == B.dim())
        result = Matrix((self.dim1,self.dim2))
        for i in range(self.dim1):
            for j in range(self.dim2):
                result[i][j] = self.A[i][j] - B[i][j]
        return result
    
    def __mul__(self, B):
        if(type(B) == int or type(B) == float):
            result = Matrix((self.dim1,self.dim2))
            for i in range(self.dim1):
                for j in range(self.dim2):
                    result[i][j] = self.A[i][j]*B
            return result
        else:
            assert(self.dim2 == B.dim1)
            result = Matrix((self.dim1,B.dim2))
            for i in range(self.dim1):
                for j in range(B.dim2):
                    for k in range(B.dim1):
                        result[i][j] += self.A[i][k]*B[k][j]
            return result
    
    def __truediv__(self,a):
        result = Matrix((self.dim1,self.dim2))
        for i in range(self.dim1):
            for j in range(self.dim2):
                result[i][j] = self.A[i][j]/a
        return result
    
    def transpose(self):
        result = Matrix((self.dim2,self.dim1))
        for i in range(self.dim1):
            for j in range(self.dim2):
                result[j][i] = self.A[i][j]
        return result
    
    def norm(self):
        result = 0
        for i in range(self.dim1):
            for j in range(self.dim2):
                result += self.A[i][j]*self.A[i][j]
        return teylor_sqrt(result)
    
    def norm_abs(self):
        result = 0
        for i in range(self.dim1):
            for j in range(self.dim2):
                result += ab(self.A[i][j])
        return result
    
    def iseye(self):
        assert(self.dim1 == self.dim2)
        for i in range(self.dim1):
            for j in range(self.dim1):
                if i == j:
                    self.A[i][j] = 1
                else:
                    self.A[i][j] = 0
                    
    def absolut(self):
        for i in range(self.dim1):
            for j in range(self.dim2):
                self.A[i][j] = abs(self.A[i][j])
        
    def det(self):
        assert(self.dim1 == self.dim2)
        result = Matrix((self.dim1,self.dim2))
        for i in range(self.dim1):
            for j in range(self.dim2):
                result[i][j] = self.A[i][j]
        max_A = result[0][0]
        column = 0
        for i in range(self.dim1-1):
            max_A = result[i][i]
            column = i
            for j in range(i,self.dim1):
                if result[j][i] > max_A:
                    max_A = result[i][j]
                    column = j
            for j in range(i,self.dim1):
                result[i][j], result[column][j] = result[column][j], result[i][j]
            #
            for v in range(i+1,self.dim1):
                koef = -(result[v][i]/result[i][i])
                for j in range(i, self.dim1):
                    result[v][j] += result[i][j]*koef
        Res = 1
        for i in range(self.dim1):
            Res *= result[i][i]
        return Res
    
    def diag(self):
        assert(self.dim1 == self.dim2)
        for i in range(self.dim1):
            S = sum(abs(A[i][j]) for j in range(self.dim2) if j != i)
            if fabs(A[i][i]) < S:
                return False
        return True
    
    def Edin(self):
        for i in range(self.dim2):
            for j in range(self.dim1):
                self.A[i][j] = 1
            
    def __str__(self):
        return str(self.A)
    
    def inv(self):
        C = [[0]*(2*self.dim1) for i in range(self.dim1)]
        for i in range(self.dim1):
            for j in range(self.dim1):
                C[i][j] = self.A[i][j]
                if (i == j): C[i][j+self.dim1] = 1
        for i in range(self.dim1):
            for j in range(self.dim1):
                if (i != j):
                    t = C[j][i]/C[i][i]
                    for k in range(2*self.dim1):
                        C[j][k] -= t*C[i][k]
        for i in range(self.dim1):
            div = C[i][i]
            for j in range(2*self.dim1):
                C[i][j] /= div
        S = Matrix((self.dim1,self.dim2))
        for i in range(self.dim1):
            for j in range(self.dim1, 2*self.dim2):
                S[i][j-self.dim1] = C[i][j]
                
        return S

In [30]:
eps = 1e-9

In [31]:
def Method_Newton(func):
    a,b = -10, 10
    n = 100
    end = True
    while end:
        end = True
        h = (b-a)/n
        xk = a
        for k in range(1,n):
            xk1 = a + k*h
            if (func(xk)*func(xk1) < 0):
                end = False
                break;
            xk = xk1
        n *= 2
    if xk < xk1:
        ak,bk = xk, xk1
    else:
        ak,bk = xk1, xk
    x = (ak+bk)/2
    while (fabs(func(x)) > eps):
        if (x < ak or x > bk):
            x = (ak+bk)/2
            if (func(x) < 0 and func(ak) < 0):
                ak = x
            else:
                bk = x
            
        x -= func(x)/f_1(x)
    return x;

In [32]:
Method_Newton(f)

-1.6983528012408105

In [33]:
#-1.69835280123 eps = 10^(-10)

In [34]:
def Jac(func,arg,x,y):
    func11 = (func(arg,x+eps,y)[0] - func(arg,x,y)[0])/eps
    func21 = (func(arg,x+eps,y)[1] - func(arg,x,y)[1])/eps
    func12 = (func(arg,x,y+eps)[0] - func(arg,x,y)[0])/eps
    func22 = (func(arg,x,y+eps)[1] - func(arg,x,y)[1])/eps
    S = Matrix((2,2))
    S[0][0] = func11
    S[0][1] = func12
    S[1][0] = func21
    S[1][1] = func22
    return S

In [35]:
def Method_Newton_for_sistem(func):
    x, y = -1.5, 0.5
    print(f'x0,y0 = {x},{y}')
    for i in range(1,10):
        x1 = x
        y1 = y
        k = 1
        asd = Sistema1(i/10,x,y)
        while (fabs(asd[0]) > eps or fabs(asd[1]) > eps):
            T = Jac(Sistema1,i/10,x,y).inv()
            SD = [sum(T[i][k]*asd[k] for k in range(2)) for i in range(2)]
            x -= SD[0]
            y -= SD[1]
            k += 1
            asd = func(x,y)
        print(f'x{i},y{i} = {x},{y}')
    return x,y

In [36]:
Method_Newton_for_sistem(Sistema)

x0,y0 = -1.5,0.5
x1,y1 = -1.703317484600364,1.346337645233534
x2,y2 = -1.7033174836544147,1.346337644576448
x3,y3 = -1.7033174844428274,1.3463376452460778
x4,y4 = -1.7033174838989829,1.346337644657042
x5,y5 = -1.7033174847838661,1.3463376446029232
x6,y6 = -1.7033174840706018,1.3463376447837372
x7,y7 = -1.7033174842310361,1.3463376442686792
x8,y8 = -1.7033174842058714,1.3463376450709008
x9,y9 = -1.7033174843433825,1.346337645168592


(-1.7033174843433825, 1.346337645168592)

In [20]:
#x1 = -1.703317484248414 eps = 10^(-9)
#y1 = 1.346337644919148 eps = 10^(-8)